In [2]:
import pandas as pd
print(pd.__version__)

1.0.2


# Decision Tree Implementation using entropy 

In [1]:
import os
from pprint import pprint
import pandas as pd
import numpy as np

header_list = ["1", "2", "3","4","5", "6", "acc"]
os.path.isfile('/Users/16786/Desktop/car.training.csv')
df= pd.read_csv('/Users/16786/Desktop/car.training.csv', names=header_list)
os.path.isfile('/Users/16786/Desktop/car.test.csv')
df2= pd.read_csv('/Users/16786/Desktop/car.test.csv', names=header_list)

df3 = pd.concat([df, df2], axis=0, sort=False)
df3.index = range(0,len(df3))

In [2]:
for x in range (0,len(df3)):
    if df3.iloc[x,6] == 'acc': 
        df3.iloc[x,6] = 0
    if df3.iloc[x,6] == 'unacc': 
        df3.iloc[x,6] = 1
df.index.drop_duplicates()
print(df3)


         1     2  3     4      5     6 acc
0    vhigh   med  2     4  small  high   0
1    vhigh   med  2     4    med  high   0
2    vhigh   med  2     4    big   med   0
3    vhigh   med  2     4    big  high   0
4    vhigh   med  2  more    med  high   0
..     ...   ... ..   ...    ...   ...  ..
995   high  high  3     2    big   med   1
996   high  high  3     2    big  high   1
997   high  high  3     4  small   low   1
998   high  high  3     4  small   med   1
999   high  high  3     4    med   low   1

[1000 rows x 7 columns]


In [3]:
#turn all strings into ints for decision tree to work with

stacked2 = df3[['1', '2', '3','4','5', '6']].stack()
df3[['1', '2', '3','4','5', '6']] = pd.Series(stacked2.factorize()[0], index=stacked2.index).unstack()
train = df3[0:800]
test = df3[801:1000]
print(test)



      1  2  3  4  5   6 acc
801  10  5  8  7  6   5   0
802  10  5  3  3  4   1   0
803  10  5  3  3  4   5   0
804  10  5  3  3  1   1   0
805  10  5  3  3  1   5   0
..   .. .. .. .. ..  ..  ..
995   5  5  8  2  6   1   1
996   5  5  8  2  6   5   1
997   5  5  8  3  4  10   1
998   5  5  8  3  4   1   1
999   5  5  8  3  1  10   1

[199 rows x 7 columns]


In [4]:
npTrain = train.to_numpy()
npTest = test.to_numpy()
print(npTest)

print(npTrain[0:5])

[[10 5 8 ... 6 5 0]
 [10 5 3 ... 4 1 0]
 [10 5 3 ... 4 5 0]
 ...
 [5 5 8 ... 4 10 1]
 [5 5 8 ... 4 1 1]
 [5 5 8 ... 1 10 1]]
[[0 1 2 3 4 5 0]
 [0 1 2 3 1 5 0]
 [0 1 2 3 6 1 0]
 [0 1 2 3 6 5 0]
 [0 1 2 7 1 5 0]]


In [5]:
x = npTrain[:, :-1] # all columns but the last
y = npTrain[:, -1] # expected to be from 0 to n_classes - 1
tx = npTest[:, :-1] # all columns but the last
ty = npTest[:, -1] # expected to be from 0 to n_classes - 1
print(tx)

[[10 5 8 7 6 5]
 [10 5 3 3 4 1]
 [10 5 3 3 4 5]
 ...
 [5 5 8 3 4 10]
 [5 5 8 3 4 1]
 [5 5 8 3 1 10]]


In [6]:
import numpy as np
import math as math
class DecisionTree:
    #the constructor you can pass max depth in the constructor
    def __init__(self, maxdepth=None):
        self.maxdepth = maxdepth
    #entropy returns the larges gain in entropy reduction which can be found on some single feature x, and the cutoff used to devide the dataset. 
    def entropy(self,x, y, etotal, predclass):
        gainMax = 0
        bestCutoff = -1
        
        #find potentail split locations, this will be done by first finding all unique values in the list, 
        #a mid point between each of these values will be found and used to make the split.
        unique = np.unique(x)
        #find the middle points between the unique values
        splits = []
        for n in range(0, unique.size-1):
            splits.append((unique[n] + unique[n+1])/2) 
        
        #preform and entropy calculation at each of the points in splits
        for n in range(0,  unique.size - 1):
            count0 = []
            count1 = []
            for m in range(0, y.size):
                if(x[m] < splits[n]):
                    count0.append(y[m])
                else:
                    count1.append(y[m])
            #now we find the lowest entropy value
            minEntropy = 1.1
            #first find out how many elements are of class 0 and 1 on each side of the tree
            c0occurrences0 = count0.count(0)
            c0occurrences1 = count0.count(1)
            
            total0 = c0occurrences0 + c0occurrences1
            c1occurrences0 = count1.count(0)
            c1occurrences1 = count1.count(1)
            total1 = c1occurrences0 + c1occurrences1
            
            if(total0 != 0 and c0occurrences0 != 0):
            #now run these numbers through the entropy equation -(c0/total)log2(c0/total) - (c1/total)log2(c1/total) alway will be less than 1
                e0 = ((c0occurrences0/total0)*math.log2(c0occurrences0/total0))* -1
            else: 
                e0 = 0
            if(total0 != 0 and c0occurrences1 != 0):
                e1 = ((c0occurrences1/total0)*math.log2(c0occurrences1/total0))* -1
            else: 
                e1 = 0
            entropy0 = e0 + e1
            if(total1 != 0 and c1occurrences0 != 0):
                e2 = ((c1occurrences0/total1)*math.log2(c1occurrences0/total1))* -1
            else: 
                e2 = 0
            if(total1 != 0 and c1occurrences1 != 0):
                e3 = ((c1occurrences1/total1)*math.log2(c1occurrences1/total1))* -1
            else: 
                e3 = 0
            entropy1 = e2 + e3
            #now that we have the entropy on both sides of the tree we can find the entropy gain of the split first find starting entropy of the node
            c0intial0 =  np.count_nonzero(y == 0)
            c0intial1 = np.count_nonzero(y == 1)
            if(c0intial0 != 0 and y.size != 0):
                e4 = ((c0intial0/y.size)*math.log2(c0intial0/y.size))* -1
            else: 
                e4 = 0
            if(c0intial1 != 0 and y.size != 0):
                e5 = ((c0intial1/y.size)*math.log2(c0intial1/y.size))* -1
            else:
                e5 = 0
            entropyOrg = e4 + e5
            #we compare the orginal entropy with the combined entropy of both sides of the split
            etotal = ((total0/y.size) * entropy0) + ((total1/y.size) * entropy1)
            #print("etotal has been made", etotal)
            gain = entropyOrg - etotal
            if(gain > gainMax):
                
                gainMax = gain
                bestCutoff = splits[n]
            if(c0intial0 >= c0intial1):
                predclass = 0
            else:
                predclass = 1
            if(n == 1):
                print('zeros at this node', c0intial0)
                print('ones at this node', c0intial1)
                print('entropy of orignal node', entropyOrg)    
                
        #math.log2()
        
        return gainMax, bestCutoff, etotal, predclass
    #this will call the entropy function on each of the features returning the feature will return the highest entropy gain and the cutoff used for that split
    def find_best_split(self, x, y, depth, etotal, predclass):
        bestFeature = -1
        cutoff = -1
        bestGain = 0
        for i in range(0, np.size(x,1)):
            tempBG, tempCF, etotal, predclass = self.entropy(x[:,i],y, etotal, predclass)
            
            if(tempBG > bestGain):
                bestGain = tempBG
                bestFeature = i
                cutoff = tempCF
                predclass = predclass
            #print(tempBG)
        return bestFeature, cutoff, predclass, etotal
    #this will call the recursive function which generates a tree that is stored in tree. 
    def fit(self, x, y):
        
        self.tree = self.generateTree(x, y,0)
    def generateTree(self, x, y, depth):
        #find out entropy information for the node using best_split function
        bestFeature, cutoff ,pred, etotal = self.find_best_split(x, y, depth, 0, -1)
        entropy = etotal
        node = Node(
            cutoff = cutoff,
            pred = pred,
            entropy = etotal,
            bestFeature = bestFeature
        )
        print('predicted class at node', pred)
        print("cut off ", cutoff)
        print("bestfeature ", bestFeature)
        print("depth ", depth)
        print('entropy of resulting nodes', etotal)
        #now split into new tree recursivly using maxdepth and no or negative infomation gain to cut off the recursion.
        if depth < self.maxdepth:
            #bestFeature will hold a value of negative one if not feature produced an information gain
            if bestFeature > -1:
                indexsL = []
                indexsR = []
                for i in range (0, y.size):
                    if(x[i,bestFeature] <= cutoff):
                        indexsL.append(i)
                    else:
                        indexsR.append(i)
                xl, yl = x[indexsL], y[indexsL]
                xr, yr = x[indexsR], y[indexsR]
                #print("indexs of new left node ", indexsL)
                
                
                depth += 1
                node.left = self.generateTree(xl, yl, depth)
                node.right = self.generateTree(xr, yr, depth)
        return node
     
    def predict(self, x):
        prediction = []
        for i in range(0,np.shape(x)[0]):
            node = self.tree
            while node.left:
            
                if x[i,node.bestFeature] < node.cutoff:
                    node = node.left
                else:
                    node = node.right
            prediction.append(node.pred)
        return prediction

In [7]:
class Node:
    def __init__(self, cutoff, entropy, pred, bestFeature):
        self.entropy = entropy
        self.pred = pred
        self.bestFeature = bestFeature
        self.cutoff = cutoff
        self.left = None
        self.right = None

# Results and trace of the trees path

In [9]:
cfl = DecisionTree(maxdepth=12)
cfl.fit(x, y)
pred = cfl.predict(tx)
print(pred)
count1 = 0
for i in range(0, ty.size):
    
    if(pred[i] == ty[i]):
        
        count1 += 1
accuracy = count1/ty.size
print("accuracy ", accuracy)

zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
zeros at this node 400
ones at this node 400
entropy of orignal node 1.0
predicted class at node 0
cut off  0.5
bestfeature  0
depth  0
entropy of resulting nodes 0.767075439252721
zeros at this node 72
ones at this node 360
entropy of orignal node 0.6500224216483541
zeros at this node 72
ones at this node 360
entropy of orignal node 0.6500224216483541
zeros at this node 72
ones at this node 360
entropy of orignal node 0.6500224216483541
zeros at this node 72
ones at this node 360
entropy of orignal node 0.6500224216483541
zeros at this node 72
ones at this node 360
entropy of orignal node 0.6500224216483541
predicted class at 

In [95]:
print(y.size)

800
